In [10]:
import os
import pandas as pd

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import *
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing

In [11]:
# def npoclass(string_input=None):

def device_selection():
    if torch.cuda.is_available():    

        print('There are %d GPU(s) available.' % torch.cuda.device_count())
        
        print('Do you want to the GPU accelaration?')

        selection = input("please enter 'yes' or 'no.'")

        if selection == 'yes':
            
            device = torch.device('cuda')
            print('Using GPU:',torch.cuda.get_device_name(0))

        elif selection =='no':
            
            print('Using the CPU instead.')
            device = torch.device("cpu")

        elif selection != 'no' or selection != 'yes':
            print("Wrong selection.\n\n")
            
            device_selection()
    
    else:
        print('No GPU available, using the CPU instead.')
        device = torch.device("cpu")
    return device
        
device = device_selection()

There are 1 GPU(s) available.
Do you want to the GPU accelaration?
please enter 'yes' or 'no.'yes
Using GPU: Quadro RTX 6000


In [ ]:
#load pretrained model
dir_path='new_classifier_bc/'
model_loaded = BertForSequenceClassification.from_pretrained(dir_path)  # re-load
tokenizer_loaded = BertTokenizer.from_pretrained(dir_path)  # re-load

In [13]:
#Test an input data
string_input=['GREENCROFT GOSHEN INC GREENCROFT GOSDEN PROVIDES ACTIVE , AFFORDABLE RETIREMENT LIVING WITH SUPPORTIVE SERVICES , ASSISTED LIVING , AND SKILLED NURSING CARE . CHARITY CARE FOR EYE JUNE 30 , 2014 AMOUNTED TO $ 4E2793922 . NURSING SERVICES - GREENCROFT GOSDEN IS HEALTH FACILITIES PROVIDE STATE-OF-THE-ART HEALTH CARE TECHNIQUES AND TECHNOLOGY IN AN INVITING AND SUPPORTIVE RESIDENTIAL ENVIRONMENT . OUR SETTING IS APPROPRIATE FOR INDIVIDUALS WHO ARE RECOVERING FROM SURGERY OR ILLNESS , NEED REHABILITATION , OR REQUIRE ONGOING LONG-TERM NURSING CARE . OUR RESTORATIVE CARE PHILOSOPHY REINFORCES THE HEALING AND REHABILITATION PROCESS AND PROMOTES INDEPENDENCE . A FULL RANGE OF ACTIVITIES , CHAPLAINCY , AND SOCIAL SERVICES ARE PROVIDED . WE PROVIDE OUTPATIENT PHYSICAL , OCCUPATIONAL , AND SPEECH/LANGUAGE THERAPIES FOR ALL AGES . WE EMPHASIZE EDUCATION AND COMMUNICATION WITH YOU , YOUR FAMILY , AND REFERRING PHYSICIANS . RESIDENTS WHO DO NOT MISUSE THEIR ASSETS MAY BE ELIGIBLE FOR SUPPORT FROM GREENCROFT COMMUNITIES FOUNDATION IF THEIR ASSETS RUN OUT . DURING THE YEAR ENDED JUNE 30 , 2014 , OVER 21 % OF TOTAL REVENUES WERE USED TO PROVIDE COMMUNITY BENEFITS , WHICH INCLUDES THE FOLLOWING AMOUNTS : INDIGENT CARE/CHARITY CARE - $ 4E2793922 , REDUCED/FREE CARE/USE OF FACILITIES - $ 4893569 , MEDICARE WRITE-OFFS - $ 12003.490 . ; DIETARY AND OTHER SERVICES - THE ORGANIZATION PROVIDES DIETARY AND OTHER MISCELLANEOUS PROGRAM SERVICES TO ITS RESIDENTS . ; RESIDENT SERVICES - GREENCROFT GOSDEN ( GG ) IS A NON-PROFIT COMMUNITY BENEFIT CORPORATION THAT PROVIDES CONTINUING CARE RETIREMENT COMMUNITY ( ICRC ) SERVICES TO 800 RESIDENTS ON A 167-ACRE CAMPUS . GG PROVIDES ICRC SERVICES INCLUDING INDEPENDENT LIVING , ASSISTED LIVING , SKILLED NURSING , ADULT DAY PROGRAMS , SENIOR CENTER PROGRAMS , AND HOME CARE SERVICES . EVERGREEN PLACE PROVIDES GG RESIDENTS WITH A VITAL OPTION IN THE CONTINUUM OF CARE IF AND WHEN THEY SHOULD NEED ASSISTANCE SERVICES . ASSISTED LIVING VACANCIES NOT FILLED BY GG ARE AVAILABLE TO OLDER ADULTS IN THE WIDER COMMUNITY . WE PARTNER WITH RESIDENTS , THEIR FAMILIES , AND OTHERS TO CREATE A COMMUNITY WHERE OLDER ADULTS ENJOY LIFE . SUCH PARTNERSHIP IS ESSENTIAL TO APPROPRIATELY MEET THE NEEDS OF A RAPIDLY GROWING OLDER ADULT POPULATION . RESIDENTS WHO DO NOT MISUSE THEIR ASSETS ARE ELIGIBLE FOR SUPPORT FROM THE GREENCROFT COMMUNITY FOUNDATION IF THEIR ASSETS RUN OUT . GG IS IN AN ACTIVE PARTNERSHIP WITH GOSDEN COLLEGE TO PROVIDE LIFELONG LEARNING OPPORTUNITIES TO PEOPLE OVER THE AGE OF 55 IN ELKHART COUNTY . ADDITIONALLY , GG PROVIDES NUMEROUS PROGRAMS AND ACTIVITIES FOR SENIORS ON AND OFF OUR CAMPUS . GG IS AFFILIATED WITH THREE HUD HOUSING ORGANIZATIONS ON ITS CAMPUS THAT ARE INTEGRATED INTO GG IS SERVICES . THESE ORGANIZATIONS SERVE 325 RESIDENTS.']

In [20]:
#preparing labels

broad_cat = ['I','II','III','IV','V','VI',
                'VII',
                'VIII',
                'IX',
                'X',
            ]

In [15]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in tqdm(string_input):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer_loaded.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 256,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

100%|██████████| 1/1 [00:00<00:00, 110.44it/s]

Original:  GREENCROFT GOSHEN INC GREENCROFT GOSDEN PROVIDES ACTIVE , AFFORDABLE RETIREMENT LIVING WITH SUPPORTIVE SERVICES , ASSISTED LIVING , AND SKILLED NURSING CARE . CHARITY CARE FOR EYE JUNE 30 , 2014 AMOUNTED TO $ 4E2793922 . NURSING SERVICES - GREENCROFT GOSDEN IS HEALTH FACILITIES PROVIDE STATE-OF-THE-ART HEALTH CARE TECHNIQUES AND TECHNOLOGY IN AN INVITING AND SUPPORTIVE RESIDENTIAL ENVIRONMENT . OUR SETTING IS APPROPRIATE FOR INDIVIDUALS WHO ARE RECOVERING FROM SURGERY OR ILLNESS , NEED REHABILITATION , OR REQUIRE ONGOING LONG-TERM NURSING CARE . OUR RESTORATIVE CARE PHILOSOPHY REINFORCES THE HEALING AND REHABILITATION PROCESS AND PROMOTES INDEPENDENCE . A FULL RANGE OF ACTIVITIES , CHAPLAINCY , AND SOCIAL SERVICES ARE PROVIDED . WE PROVIDE OUTPATIENT PHYSICAL , OCCUPATIONAL , AND SPEECH/LANGUAGE THERAPIES FOR ALL AGES . WE EMPHASIZE EDUCATION AND COMMUNICATION WITH YOU , YOUR FAMILY , AND REFERRING PHYSICIANS . RESIDENTS WHO DO NOT MISUSE THEIR ASSETS MAY BE ELIGIBLE FOR SUP

In [16]:
#prepare dataloader
batch_size = 32
test_data = TensorDataset(input_ids, attention_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [17]:
import numpy as np
import time
import datetime
import random

#setup time
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [18]:
model_loaded.cuda()

logits_all=[]
label_ids_all=[]

t0 = time.time()

for batch in test_dataloader:

    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)


    b_input_ids, b_input_mask = batch


    with torch.no_grad():        

        outputs = model_loaded(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=b_input_mask)


    logits = outputs[0]


    logits = logits.detach().cpu().numpy()
    logits_all+=list(np.argmax(logits, axis=1))


    nb_eval_steps += 1

print("  Validation took: {:}".format(format_time(time.time() - t0)))

  Validation took: 0:00:00


In [21]:
logits_all_letter=preprocessing.LabelEncoder().fit(broad_cat).inverse_transform(logits_all)

In [24]:
df_eval=pd.DataFrame([logits_all_letter]).T.rename(columns={0:'pred', 1:'true'})

In [25]:
df_eval

,pred
0,V
